In [1]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch.nn.modules.loss")
import utils

import argparse
import inspect
import importlib


class CFG6:
    DEBUG = True
    note_num = "StackingOpt/LGBM003/CFG6"
    n_splits = 2  #データの分割
    seed = 42
    
    #model
    lgb_params = {
                'boosting_type': 'gbdt',
                'objective': 'regression',
                'metric': 'mae',
                'num_boost_round':10,
                'early_stopping_rounds':100,
                'num_leaves': 31,
                'learning_rate': 0.05,
                'feature_fraction': 0.9,
                'bagging_fraction': 0.8,
                'bagging_freq': 5,
                'verbose': 5,
                'device_type': 'gpu',
                'seed':42
                }
    
    #日付
    train_start_date = "201406010000"
    train_end_date = "201406070000"
    test_start_date = "201406070000"
    test_end_date = "201406080000"

    #特徴量
    interpolated_dir = "H:\study\output\StackingOpt\EDA005"
    pred_dir = "H:\study\output\StackingOpt\EDA006"
    flo_dir = 'H:\study\output\StackingOpt\EDA006'
    flo_unique_dir = 'H:/study/output/StackingOpt/EDA006/'

    features = ['two_weeks_max', 'id', 'month_sin', 'month_cos', 'day_sin', 'day_cos', 'hour_sin', 'hour_cos', 'year_sin', 'year_cos',\
                'prev_30m_generation', 'id_lat', 'id_lng', 'id_lat_mesh', 'id_lng_mesh', 'pvrate', 'observed_max']

    use_interpolated_features = True
    use_pred_features = True
    use_flo_features = True
    use_flo_lat_features = True
    use_flo_lon_features = True
    use_flo_unique_features = True #オプティカルフローのunique_idの予測値

    target = 'nv2'

    #oofで保存するcol
    saved_cols = ["datetime","id","fold","observed_max","generation",target,"pred"]


def main(cfg):
    
    if cfg.DEBUG:
        OUTPUT_DIR = f'H:/study/output/DEBUG/{cfg.note_num}/'
    else:
        OUTPUT_DIR = f'H:/study/output/{cfg.note_num}/'

    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)

    utils.set_seeds()
    # 時系列の分割設定
    train_date_list = utils.create_time_series_data(cfg.train_start_date,cfg.train_end_date)
    train_date_list_split = np.array_split(train_date_list, cfg.n_splits)

    test_dates = utils.create_time_series_data(cfg.test_start_date,cfg.test_end_date)

    #データセットの読み込み
    unique_id = ['10000095', '10000269', '1020000002', '1110000001', '1110000010', '1110000011', '1110000012', '1110000013', '1110000014', '1110000015', '1160000025', '1160000090', '1160000091', '1160000182', '1160000185', '1160000253', '1160000387', '1160000402', '1160000419', '1160000420', '1160000423', '1270000026', '1280000048', '1550000001', '1650000004', '1680000001', '1680000002', '1680000003', '1680000004', '1680000010', '1680000017', '1680000021', '1680000033', '1680000047', '1680000054', '1680000057', '1680000063', '1680000067', '1680000080', '1680000081', '1680000097', '1680000107', '1680000108', '1680000112', '1680000151', '1680000152', '1680000213', '1680000216', '1680000217', '1680000218', '1680000223', '1680000228', '1680000285', '1680000287', '1680000327', '1680000364', '2220000001', '2220000002', '2220000003', '2730000001', '2910000002', '3000000007', '3000000012', '3000000042', '5000000044', '5000000045', '6000000016', '6000000017', '6060000016', '6060000017', '6060000018', '6170000016', '6170000123', '6170000124', '6170000125', '6620000065', '6620000066', '6620000088', '6620000089', '6620000111', '6620000117', '6620000118', '6620000121', '6620000122', '6620000123', '6620000124', '6620000131', '6620000132', '6910000180', '6910000198', '6910000200', '6910000206', '6910000216', '6910000217', '6910000239', '6910000240', '6910000249', '6910000250', '6910000421', '6910000424', '6910000425', '6910000469', '6910000470']
    unique_id = [int(i) for i in unique_id]
    to_unique_id = [str(num).zfill(10) for num in unique_id]
    df = utils.get_preprocessing_data(to_unique_id)
    df.drop_duplicates(inplace=True)
    df.drop_duplicates(subset=["id","datetime"],inplace=True)
    df = df.groupby('id').apply(utils.prev_30m_generation).reset_index(level=0, drop=True)
    id_all_data = pd.read_csv("H:\study\output\StackingOpt\EDA006\id_all_data.csv")
    df = df.merge(id_all_data,on=["id"],how="left")
    df.dropna(subset=["year"],inplace=True) #utils.prev_30m_generationで30分間隔のデータセットになっているため欠損が出ている。
    df["nv2"] = df["generation"] / df["observed_max2"]

    #oof作成用
    df["pred"] = 0
    df.loc[df.datetime.isin(test_dates),"fold"] = "test"

    for fold in range(len(train_date_list_split)):
        print(f"\nFold {fold + 1}")
        train_dates = np.concatenate(train_date_list_split[:fold] + train_date_list_split[fold+1:])
        valid_dates = train_date_list_split[fold]

        X_train, y_train = df.loc[df.datetime.isin(train_dates),cfg.features+["datetime"]],df.loc[df.datetime.isin(train_dates),cfg.target]
        X_valid, y_valid = df.loc[df.datetime.isin(valid_dates),cfg.features+["datetime"]],df.loc[df.datetime.isin(valid_dates),cfg.target]
        X_test, y_test = df.loc[df.datetime.isin(test_dates),cfg.features+["datetime"]],df.loc[df.datetime.isin(test_dates),cfg.target]
        df.loc[df.datetime.isin(valid_dates),"fold"] = fold

        if cfg.use_interpolated_features:
            X_train_interpolated, y_train_interpolated = utils.get_interpolated_mesh_data(cfg.interpolated_dir,train_dates)
            X_valid_interpolated, y_valid_interpolated = utils.get_interpolated_mesh_data(cfg.interpolated_dir,valid_dates)
            X_test_interpolated, y_test_interpolated = utils.get_interpolated_mesh_data(cfg.interpolated_dir,test_dates)
            
            X_train_interpolated = pd.DataFrame(X_train_interpolated,columns=[f"{i}_interpolated" for i in range(X_train_interpolated.shape[1])])
            X_train_interpolated["datetime"] = pd.to_datetime(train_dates, format="%Y%m%d%H%M")

            X_valid_interpolated = pd.DataFrame(X_valid_interpolated,columns=[f"{i}_interpolated" for i in range(X_valid_interpolated.shape[1])])
            X_valid_interpolated["datetime"] = pd.to_datetime(valid_dates, format="%Y%m%d%H%M")

            X_test_interpolated = pd.DataFrame(X_test_interpolated,columns=[f"{i}_interpolated" for i in range(X_test_interpolated.shape[1])])
            X_test_interpolated["datetime"] = pd.to_datetime(test_dates, format="%Y%m%d%H%M")

            X_train = X_train.merge(X_train_interpolated,on=["datetime"],how="left")
            X_valid = X_valid.merge(X_valid_interpolated,on=["datetime"],how="left")
            X_test = X_test.merge(X_test_interpolated,on=["datetime"],how="left")

        if cfg.use_pred_features:
            X_train_interpolated_pred = utils.get_pred_interpolated_mesh_data(cfg.pred_dir,train_dates)
            X_valid_interpolated_pred = utils.get_pred_interpolated_mesh_data(cfg.pred_dir,valid_dates)
            X_test_interpolated_pred = utils.get_pred_interpolated_mesh_data(cfg.pred_dir,test_dates)
            
            X_train_interpolated_pred  = pd.DataFrame(X_train_interpolated_pred,columns=[f"{i}_pred" for i in range(X_train_interpolated_pred.shape[1])])
            X_train_interpolated_pred ["datetime"] = pd.to_datetime(train_dates, format="%Y%m%d%H%M")

            X_valid_interpolated_pred = pd.DataFrame(X_valid_interpolated_pred,columns=[f"{i}_pred" for i in range(X_valid_interpolated_pred.shape[1])])
            X_valid_interpolated_pred["datetime"] = pd.to_datetime(valid_dates, format="%Y%m%d%H%M")

            X_test_interpolated_pred = pd.DataFrame(X_test_interpolated_pred,columns=[f"{i}_pred" for i in range(X_test_interpolated_pred.shape[1])])
            X_test_interpolated_pred["datetime"] = pd.to_datetime(test_dates, format="%Y%m%d%H%M")

            X_train = X_train.merge(X_train_interpolated_pred,on=["datetime"],how="left")
            X_valid = X_valid.merge(X_valid_interpolated_pred,on=["datetime"],how="left")
            X_test = X_test.merge(X_test_interpolated_pred,on=["datetime"],how="left")

        if cfg.use_flo_features:
            X_train_flo= utils.get_flo_mesh_data(cfg.pred_dir,train_dates)
            X_valid_flo= utils.get_flo_mesh_data(cfg.pred_dir,valid_dates)
            X_test_flo= utils.get_flo_mesh_data(cfg.pred_dir,test_dates)
            
            X_train_flo = pd.DataFrame(X_train_flo,columns=[f"{i}_flo" for i in range(X_train_flo.shape[1])])
            X_train_flo["datetime"] = pd.to_datetime(train_dates, format="%Y%m%d%H%M")

            X_valid_flo= pd.DataFrame(X_valid_flo,columns=[f"{i}_flo" for i in range(X_valid_flo.shape[1])])
            X_valid_flo["datetime"] = pd.to_datetime(valid_dates, format="%Y%m%d%H%M")

            X_test_flo= pd.DataFrame(X_test_flo,columns=[f"{i}_flo" for i in range(X_test_flo.shape[1])])
            X_test_flo["datetime"] = pd.to_datetime(test_dates, format="%Y%m%d%H%M")

            X_train = X_train.merge(X_train_flo,on=["datetime"],how="left")
            X_valid = X_valid.merge(X_valid_flo,on=["datetime"],how="left")
            X_test = X_test.merge(X_test_flo,on=["datetime"],how="left")

        if cfg.use_flo_lat_features:
            X_train_flo_lat= utils.get_flo_lat_mesh_data(cfg.pred_dir,train_dates)
            X_valid_flo_lat= utils.get_flo_lat_mesh_data(cfg.pred_dir,valid_dates)
            X_test_flo_lat= utils.get_flo_lat_mesh_data(cfg.pred_dir,test_dates)
            
            X_train_flo_lat = pd.DataFrame(X_train_flo_lat,columns=[f"{i}_flo_lat" for i in range(X_train_flo_lat.shape[1])])
            X_train_flo_lat["datetime"] = pd.to_datetime(train_dates, format="%Y%m%d%H%M")

            X_valid_flo_lat= pd.DataFrame(X_valid_flo_lat,columns=[f"{i}_flo_lat" for i in range(X_valid_flo_lat.shape[1])])
            X_valid_flo_lat["datetime"] = pd.to_datetime(valid_dates, format="%Y%m%d%H%M")

            X_test_flo_lat= pd.DataFrame(X_test_flo_lat,columns=[f"{i}_flo_lat" for i in range(X_test_flo_lat.shape[1])])
            X_test_flo_lat["datetime"] = pd.to_datetime(test_dates, format="%Y%m%d%H%M")

            X_train = X_train.merge(X_train_flo_lat,on=["datetime"],how="left")
            X_valid = X_valid.merge(X_valid_flo_lat,on=["datetime"],how="left")
            X_test = X_test.merge(X_test_flo_lat,on=["datetime"],how="left")

        if cfg.use_flo_lon_features:
            X_train_flo_lon= utils.get_flo_lon_mesh_data(cfg.pred_dir,train_dates)
            X_valid_flo_lon= utils.get_flo_lon_mesh_data(cfg.pred_dir,valid_dates)
            X_test_flo_lon= utils.get_flo_lon_mesh_data(cfg.pred_dir,test_dates)
            
            X_train_flo_lon = pd.DataFrame(X_train_flo_lon,columns=[f"{i}_flo_lon" for i in range(X_train_flo_lon.shape[1])])
            X_train_flo_lon["datetime"] = pd.to_datetime(train_dates, format="%Y%m%d%H%M")

            X_valid_flo_lon= pd.DataFrame(X_valid_flo_lon,columns=[f"{i}_flo_lon" for i in range(X_valid_flo_lon.shape[1])])
            X_valid_flo_lon["datetime"] = pd.to_datetime(valid_dates, format="%Y%m%d%H%M")

            X_test_flo_lon= pd.DataFrame(X_test_flo_lon,columns=[f"{i}_flo_lon" for i in range(X_test_flo_lon.shape[1])])
            X_test_flo_lon["datetime"] = pd.to_datetime(test_dates, format="%Y%m%d%H%M")

            X_train = X_train.merge(X_train_flo_lon,on=["datetime"],how="left")
            X_valid = X_valid.merge(X_valid_flo_lon,on=["datetime"],how="left")
            X_test = X_test.merge(X_test_flo_lon,on=["datetime"],how="left")

        if cfg.use_flo_unique_features:
            X_train_flo_unique = utils.get_unique_pred_interpolated(cfg.flo_unique_dir,train_dates,unique_id)
            X_valid_flo_unique = utils.get_unique_pred_interpolated(cfg.flo_unique_dir,valid_dates,unique_id)
            X_test_flo_unique = utils.get_unique_pred_interpolated(cfg.flo_unique_dir,test_dates,unique_id)
            
            X_train_flo_unique = pd.DataFrame(X_train_flo_unique,columns=[f"{i}_flo_unique" for i in range(X_train_flo_unique.shape[1])])
            X_train_flo_unique["datetime"] = pd.to_datetime(train_dates, format="%Y%m%d%H%M")

            X_valid_flo_unique= pd.DataFrame(X_valid_flo_unique,columns=[f"{i}_flo_unique" for i in range(X_valid_flo_unique.shape[1])])
            X_valid_flo_unique["datetime"] = pd.to_datetime(valid_dates, format="%Y%m%d%H%M")

            X_test_flo_unique= pd.DataFrame(X_test_flo_unique,columns=[f"{i}_flo_unique" for i in range(X_test_flo_unique.shape[1])])
            X_test_flo_unique["datetime"] = pd.to_datetime(test_dates, format="%Y%m%d%H%M")

            X_train = X_train.merge(X_train_flo_unique,on=["datetime"],how="left")
            X_valid = X_valid.merge(X_valid_flo_unique,on=["datetime"],how="left")
            X_test = X_test.merge(X_test_flo_unique,on=["datetime"],how="left")


        X_train.drop("datetime",axis=1,inplace=True)
        X_valid.drop("datetime",axis=1,inplace=True)
        X_test.drop("datetime",axis=1,inplace=True)

        # Train LightGBM model
        model = utils.train_lgbm(X_train, y_train, X_valid, y_valid, cfg.lgb_params)
        save_path = OUTPUT_DIR + f"/lgbm_fold{fold}.txt"
        model.save_model(save_path)

        # Evaluate model
        valid_preds = model.predict(X_valid, num_iteration=model.best_iteration)
        mse = utils.compute_mse(y_valid, valid_preds)
        mae = utils.compute_mae(y_valid, valid_preds)
        print(f"Fold {fold + 1} MSE: {mse}, MAE: {mae}")

        # Make predictions for the test set
        test_preds = model.predict(X_test, num_iteration=model.best_iteration)

        df.loc[df.datetime.isin(valid_dates),"pred"] = valid_preds
        df.loc[df.datetime.isin(test_dates),"pred"] += test_preds

    df.loc[df.datetime.isin(test_dates),"pred"] /= len(train_date_list_split)
    df.loc[df.datetime.isin(train_date_list+test_dates),cfg.saved_cols].to_csv(OUTPUT_DIR+"oof.csv",index=False)

    oof_mse = utils.compute_mse(df.loc[df.datetime.isin(train_date_list),cfg.target] , df.loc[df.datetime.isin(train_date_list),"pred"])
    test_mse = utils.compute_mse(df.loc[df.datetime.isin(test_dates),cfg.target] , df.loc[df.datetime.isin(test_dates),"pred"])

    oof_mae = utils.compute_mae(df.loc[df.datetime.isin(train_date_list),cfg.target] , df.loc[df.datetime.isin(train_date_list),"pred"])
    test_mae = utils.compute_mae(df.loc[df.datetime.isin(test_dates),cfg.target] , df.loc[df.datetime.isin(test_dates),"pred"])

    print('-'*40)
    print(f"Overall Out-of-Fold RMSE: {np.sqrt(oof_mse):.4f}")
    print(f"Overall Out-of-Fold MAE: {oof_mae:.4f}")
    print()
    print(f"Overall Test RMSE: {np.sqrt(test_mse):.4f}")
    print(f"Overall Test MAE: {test_mae:.4f}")
    print('-'*40)


In [2]:
main(CFG6)


Fold 1


c:\Users\Yosui\.conda\envs\lab\Lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\Yosui\.conda\envs\lab\Lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 75257
[LightGBM] [Info] Number of data points in the train set: 6777, number of used features: 1078
[LightGBM] [Info] Using GPU Device: Quadro M5000, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 1078 dense feature groups (6.98 MB) transferred to GPU in 0.067552 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 0.213732
[LightGBM] [Debug] Re-bagging, using 5379 data to train
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[1]	valid_0's l1: 0.333745
Training until validation scores don't improve for 100 rounds
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[2]	valid_0's l1: 0.325145
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[3]	valid_0's l1: 0.317154
[LightGBM] [Debug] Trained a tre

c:\Users\Yosui\.conda\envs\lab\Lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\Yosui\.conda\envs\lab\Lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 75046
[LightGBM] [Info] Number of data points in the train set: 6818, number of used features: 1078
[LightGBM] [Info] Using GPU Device: Quadro M5000, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 1078 dense feature groups (7.02 MB) transferred to GPU in 0.067387 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 0.493143
[LightGBM] [Debug] Re-bagging, using 5410 data to train
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[1]	valid_0's l1: 0.317572
Training until validation scores don't improve for 100 rounds
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[2]	valid_0's l1: 0.310422
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[3]	valid_0's l1: 0.304752
[LightGBM] [Debug] Trained a tree